In [1]:
"""
- 새로운 Jupyter notebook에서 리서치 AI 에이전트를 만들고 커스텀 도구를 부여합니다.
- 에이전트는 다음 작업을 수행할 수 있어야 합니다:
    - Wikipedia에서 검색
    - DuckDuckGo에서 검색
    - 웹사이트의 텍스트를 스크랩하고 추출합니다.
    - 리서치 결과를 .txt 파일에 저장하기
    - 다음 쿼리로 에이전트를 실행합니다: "Research about the XZ backdoor" 라는 쿼리로 에이전트를 실행하면, 에이전트는 Wikipedia 또는 DuckDuckGo에서 검색을 시도하고, DuckDuckGo에서 웹사이트를 찾으면 해당 웹사이트에 들어가서 콘텐츠를 추출한 다음 .txt 파일에 조사 내용을 저장하는 것으로 완료해야 합니다.
"""

from langchain.schema import SystemMessage
import streamlit as st
import os
import requests
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper, WikipediaAPIWrapper

In [2]:
llm = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo-1106")

class WikipediaSearchTool(BaseTool):
    name = "WikipediaSearchTool"
    description = "Use this tool to search Wikipedia for a given query."

    class Args(BaseModel):
        query: str = Field(description="Query to search on Wikipedia.")

    args_schema: Type[Args] = Args

    def _run(self, query):
        api = WikipediaAPIWrapper()
        return api.run(query)

class DuckDuckGoSearchTool(BaseTool):
    name = "DuckDuckGoSearchTool"
    description = "Use this tool to search DuckDuckGo for a given query."

    class Args(BaseModel):
        query: str = Field(description="Query to search on DuckDuckGo.")

    args_schema: Type[Args] = Args

    def _run(self, query):
        api = DuckDuckGoSearchAPIWrapper()
        return api.run(query)

class WebScraperTool(BaseTool):
    name = "WebScraperTool"
    description = "Scrapes content from a given URL."

    class Args(BaseModel):
        url: str = Field(description="URL to scrape content from.")

    args_schema: Type[Args] = Args

    def _run(self, url):
        response = requests.get(url)
        return response.text

class TextSaverTool(BaseTool):
    name = "TextSaverTool"
    description = "Saves text to a file."

    class Args(BaseModel):
        text: str = Field(description="Text to save.")
        filename: str = Field(default="./Users/jihoonchoi/Projects/GPTs/fullstackgpt/research_result.txt", description="Filename to save text to.")

    args_schema: Type[Args] = Args

    def _run(self, text, filename):
        with open(filename, 'w') as file:
            file.write(text)
        return f"Text saved to {filename}"

In [3]:
agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        WikipediaSearchTool(),
        DuckDuckGoSearchTool(),
        WebScraperTool(),
        TextSaverTool(),
    ],
)

query = "Research about the XZ backdoor"
result = agent.invoke(query)
print(result['output'])



> Entering new AgentExecutor chain...

Invoking: `WikipediaSearchTool` with `{'query': 'XZ backdoor'}`


Page: XZ Utils backdoor
Summary: On 29 March 2024, software developer Andres Freund reported that he had found a maliciously introduced backdoor in the Linux utility xz within the liblzma library in versions 5.6.0 and 5.6.1 released in February 2024.While xz is commonly present in most Linux distributions, the backdoor only targeted Debian- and RPM-based systems running on the x86-64 architecture. At the time of discovery the backdoored version had not yet been widely deployed to production systems, but was present in development versions of major distributions.The backdoor gives an attacker who possesses a specific Ed448 private key remote code execution capabilities on the affected Linux system. The issue has been assigned a CVSS score of 10.0, the highest possible score.



Page: XZ Utils
Summary: XZ Utils (previously LZMA Utils) is a set of free software command-line lossless 